In [376]:
import numpy as np
# import matplotlib as plt
import PIL.Image as img
from pathlib import Path
import math
working_dir = Path().resolve()
print(working_dir)

C:\Users\Vexrina\Desktop\projects\neuron_labs


In [377]:
def convert(file_name):
    '''
    Перевод png файла в массив тюплов RGBA
    (red green blue alpha)
    '''
    image = img.open(working_dir/file_name)
    return image.convert('RGBA').getdata()

In [378]:
def take_datas(letters):
    '''
    Перевод всех изображений в матрицу и сохранений всех матриц в отдельный массив
    '''
    letters_data = []
    for letter in letters:
        for i in range(4): # 4 - количество образов буквы
            data = convert(f'letters/{letter}{i+1}.png')
            letters_data.append(data)
    return letters_data

In [379]:
letters = ['q','h','i','w'] # буквы, для которых составлены образы

datas = take_datas(letters)
# print(len(datas))
def tuples_to_mtx(data:list[tuple])->list[list[int]]:
    '''
    Конвентирует массив тюплов в матрицу, 
    где ТОЛЬКО черный цвет с alpha=255 - единица,
    а любое другое значение - 0
    '''
    result = [[]]
    i, j = 0, 0
    for item in data:
        if j == 32:
            result.append([])
            i+=1
            j=0
        if item[0] == 0 and item[1] == 0 and item[2] == 0 and item[3] == 255:
            result[i].append(1)
            j+=1
        else:
            result[i].append(0)
            j+=1
    return result


In [380]:
mtxs=[]
for data in datas:
    mtxs.append(tuples_to_mtx(data))

In [381]:
def print_mtx(mtx):
    for row in mtx:
        print(row)

def mtx_to_vector(mtx):
    result = []
    for row in mtx:
        for item in row:
            result.append(item)
    return result

In [382]:
class Neuron:
    def sigmoid(self, x: float):
        '''
        функция активации, посоветовал знакомый ее
        другой знакомый, сказал что лучше накатить софтмакс
        '''
        return 1 / (1 + math.exp(-x))

    def softmax(self, array):
        e_x = np.exp(array-np.max(array))
        return e_x / e_x.sum(axis=0)

    def __init__(
            self, vectors, responses, epoch: int,
            speed: float = np.random.uniform(low=0.05, high=1)
    ):
        '''
        инициализация класса
        Веса в диапазоне -0.03 до 0.03, матрица размером 4х1024
        буквы - входные вектора буков
        эпохи - эпохи
        ответы - входные вектора ответов
        скорость - рандомное число в диапазоне между 0.05 до 1.
        '''
        self.weight = np.random.uniform(low=-0.03, high=0.03, size=(4, 1024))
        self.letters = vectors
        self.epoch = epoch
        self.responses = responses
        self.speed = speed
        print(f'Your setup:\nepoch - {epoch}\nspeed - {speed}')

    def weight_sum(self, letter):
        '''
        Высчитываю произведение столбцов матрицы весов на входной вектор.
        Если значение суммы таких произведений >0 то в функцию активации
        передаю 1, в противном случае 0.
        Высчитываю вектор "ответа" с помощью сигмоиды.
        '''
        self.sum_multiply = [0, 0, 0, 0]
        for index in range(1024):
            self.sum_multiply[0] += (self.weight[0][index]*letter[index])
            self.sum_multiply[1] += (self.weight[1][index]*letter[index])
            self.sum_multiply[2] += (self.weight[2][index]*letter[index])
            self.sum_multiply[3] += (self.weight[3][index]*letter[index])

        for i in range(4):
            if self.sum_multiply[i] > 0:
                self.sum_multiply[i] = 1
            else:
                self.sum_multiply[i] = 0

        self.activate = self.softmax(self.sum_multiply)

    def correction(self, index):
        self.e = np.array([i-j for i, j in zip(self.responses[index], self.activate)])
        self.delta = self.speed*self.e
        self.delta_weight = np.outer(self.delta,self.letters[index])
        self.weight = self.weight+self.delta_weight

    def train(self):
        '''
        тестовый метод для тренировки
        '''
        # for __ in range(self.epoch):
        epoch = 0
        while (True):
            epoch += 1
            for index in range(4):
                self.weight_sum(self.letters[index])
                if self.activate[index] != self.responses[index][index]:
                    self.correction(index)
            if epoch % 1000 == 0:
                print(f'epoch - {epoch}')
                for index in range(4):
                    self.weight_sum(self.letters[index])
                    print(self.activate)
                
            if epoch % 5000 ==0:
                break

In [383]:
vector = []
response = []
for i in range(0,16,4):
    vector.append(mtx_to_vector(mtxs[i]))

for i in range(4):
    temp = []
    for j in range(4):
        if j==i:
            temp.append(1)
        else:
            temp.append(0)
    response.append(temp)


N = Neuron(vectors=vector, responses=response, epoch=500)

import time

N.train()

Your setup:
epoch - 500
speed - 0.8787312842498238
epoch - 1000
[0.47536689 0.1748777  0.1748777  0.1748777 ]
[0.1748777  0.47536689 0.1748777  0.1748777 ]
[0.1748777  0.1748777  0.47536689 0.1748777 ]
[0.1748777  0.1748777  0.1748777  0.47536689]
epoch - 2000
[0.47536689 0.1748777  0.1748777  0.1748777 ]
[0.1748777  0.47536689 0.1748777  0.1748777 ]
[0.1748777  0.1748777  0.47536689 0.1748777 ]
[0.1748777  0.1748777  0.1748777  0.47536689]
epoch - 3000
[0.47536689 0.1748777  0.1748777  0.1748777 ]
[0.1748777  0.47536689 0.1748777  0.1748777 ]
[0.1748777  0.1748777  0.47536689 0.1748777 ]
[0.1748777  0.1748777  0.1748777  0.47536689]
epoch - 4000
[0.47536689 0.1748777  0.1748777  0.1748777 ]
[0.1748777  0.47536689 0.1748777  0.1748777 ]
[0.1748777  0.1748777  0.47536689 0.1748777 ]
[0.1748777  0.1748777  0.1748777  0.47536689]
epoch - 5000
[0.47536689 0.1748777  0.1748777  0.1748777 ]
[0.1748777  0.47536689 0.1748777  0.1748777 ]
[0.1748777  0.1748777  0.47536689 0.1748777 ]
[0.1748777